In [1]:
import pandas as pd
from loguru import logger
# Setup environment for running MATLAB code if not done externally
import os
%load_ext autoreload

# os.environ["MR"] = f"{os.environ['HOME']}/PSA/MATLAB_runtime/R2023b"
os.environ["MR"] = f"{os.environ['HOME']}/MATLAB/R2023b"
MR = os.environ["MR"]
os.environ["LD_LIBRARY_PATH"] = f"{MR}/runtime/glnxa64:{MR}/bin/glnxa64:{MR}/sys/os/glnxa64:{MR}/sys/opengl/lib/glnxa64"


In [2]:
# Visualization packages

from pathlib import Path
import os
import hjson
import json
import numpy as np
import pandas as pd
import time
from loguru import logger
from phd_visualizations import save_figure
from phd_visualizations.constants import generate_plotly_config
from phd_visualizations.test_timeseries import experimental_results_plot

resample_figures = False

# Visualize result so far
with open( Path("data") / 'variables_config.hjson') as f:
    vars_config = hjson.load(f)

# Load plot configuration
with open( Path("data") / "plot_config.hjson") as f:
    plt_config = hjson.load(f)

In [3]:
# TODO: Actualizar `data` para que tenga un tiempo de muestreo de 10 segundos

df = pd.read_csv('data/dev.csv', parse_dates=['time'], index_col='time')

In [6]:
from importlib import reload
from solarMED_modeling.solar_med import SolarMED
# reload(SolarMED)
# from solarMED_modeling.solar_med import SolarMED

%autoreload 2

idx_start = 500
span = 70
idx_end = 1500
df_mod = pd.DataFrame()

# Initialize model
model = SolarMED(
    sample_time=10,
    curve_fits_path='data/curve_fits.json',
    
    # Initial states
    ## Thermal storage
    Tts_h=[df['Tts_h_t'].iloc[idx_start], df['Tts_h_m'].iloc[idx_start], df['Tts_h_b'].iloc[idx_start]], 
    Tts_c=[df['Tts_c_t'].iloc[idx_start], df['Tts_c_m'].iloc[idx_start], df['Tts_c_b'].iloc[idx_start]],
    
    ## Solar field
    Tsf_in_ant=df['Tsf_in'].iloc[idx_start-span:idx_start].values,
    msf_ant=df['qsf'].iloc[idx_start-span:idx_start].values,
    
    cost_w = 3, # €/m³ 
    cost_e = 0.05 # €/kWhe
)

# Run model
for idx in range(idx_start, idx_end):
    # idx = 1
    ds = df.iloc[idx]
    # df.to_csv('data/dev.csv')
        
    logger.info(f"Iteration {idx} / {len(df)}, last operation state was {model.operating_state}")
    start_time = time.time()
    
    model.step(
        # Decision variables
        ## MED
        mmed_s=ds['qmed_s'],
        mmed_f=ds['qmed_f'],
        Tmed_s_in=ds['Tmed_s_in'],
        Tmed_c_out=ds['Tmed_c_out'],
        ## Thermal storage
        mts_src=ds['qts_src'],
        ## Solar field
        Tsf_out=ds['Tsf_out'],
        
        # Inputs
        # When the solar field is starting up, a flow can be provided to sync the model with the real system, if a valid Tsf_out is provided, it will be prioritized
        msf=ds['qsf'] if ds['qsf'] > 4 else None,
        
        # Environment variables
        Tmed_c_in=ds['Tmed_c_in'],
        Tamb=ds['Tamb'],
        I=ds['I'],
    )
    
    logger.info(f"Elapsed time: {time.time()-start_time:.2f} seconds")
    logger.info(f'Tsf,in,mod = {model.Tsf_in:.2f}, Tsf,in,meas = {ds["Tsf_in"]:.2f}, msf_meas = {ds["qsf"]:.2f}, msf_mod = {model.msf:.2f}')
    
    df_mod = model.to_dataframe(df_mod)

In [1]:
# Sync model index with measured data
df_mod.index = df.index[idx_start:idx if idx<idx_end-1 else idx_end]

fig = experimental_results_plot(plt_config, df, df_comp=df_mod, vars_config=vars_config, resample=resample_figures)

fig.show(
    config=generate_plotly_config(fig, figure_name=f'solar_med_{df.index[0].strftime("%Y%m%d")}')
)